In [1]:
import pandas as pd

In [2]:
def read_in_chunks(file_path, sep='\t'):
    chunk_size = 1000
    chunks = pd.read_csv(file_path, sep=sep, chunksize=chunk_size)
    df = pd.concat(chunks)
    return df

In [3]:
df1 = read_in_chunks('Data/name.basics.tsv')
df1.head(3)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13510205 entries, 0 to 13510204
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   nconst             object
 1   primaryName        object
 2   birthYear          object
 3   deathYear          object
 4   primaryProfession  object
 5   knownForTitles     object
dtypes: object(6)
memory usage: 618.4+ MB


In [4]:
df1.replace('\\N', pd.NA, inplace=True)

In [6]:
df1.isnull().sum()

nconst                      0
primaryName                56
birthYear            12894909
deathYear            13281540
primaryProfession     2594827
knownForTitles        1492307
dtype: int64

In [5]:
df1['primaryProfession'] = df1['primaryProfession'].apply(lambda x: x.split(',') if pd.notna(x) else [])

In [6]:
df1['knownForTitles'] = df1['knownForTitles'].apply(lambda x: x.split(',') if pd.notna(x) else [])

In [7]:
df1 = df1.rename(columns={'nconst': 'id'})

In [18]:
df1.head(3)

,id,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"[actor, miscellaneous, producer]","[tt0072308, tt0050419, tt0053137, tt0027125]"
1,nm0000002,Lauren Bacall,1924,2014,"[actress, soundtrack, archive_footage]","[tt0037382, tt0075213, tt0117057, tt0038355]"
2,nm0000003,Brigitte Bardot,1934,<NA>,"[actress, music_department, producer]","[tt0057345, tt0049189, tt0056404, tt0054452]"


In [8]:
from neo4j import GraphDatabase, RoutingControl, basic_auth

In [9]:
# DB URI and authentication
URI = "bolt://localhost:7687"
AUTH = ("neo4j","12345678")

In [10]:
driver = GraphDatabase.driver(URI, auth=AUTH)

In [11]:
driver.verify_connectivity()

In [12]:
# template write request
def write_query(query, params={}):
    with driver.session(database="neo4j") as session:
        session.execute_write(lambda tx: tx.run(query, params).data())

In [13]:
create_actor_query = """
MERGE (a:Actor {id: $id})
SET a.name = CASE trim(toString($name)) WHEN 'pd.NA' THEN null ELSE $name END, 
    a.birthYear = CASE trim(toString($birthYear)) WHEN 'pd.NA' THEN null ELSE $birthYear END, 
    a.deathYear = CASE trim(toString($deathYear)) WHEN 'pd.NA' THEN null ELSE $deathYear END
"""

In [14]:
create_profession_query = """
MERGE (p:Profession {name: $profession})
"""

In [15]:
create_known_for_title_query = """
MERGE (m:Movie {id: $titleId})
"""

In [16]:
create_actor_profession_relationship = """
MATCH (a:Actor {id: $actorId}), (p:Profession {name: $profession})
MERGE (a)-[:IS_A]->(p)
"""

In [17]:
create_actor_title_relationship = """
MATCH (a:Actor {id: $actorId}), (m:Movie {id: $titleId})
MERGE (a)-[:KNOWN_FOR]->(m)
"""

In [18]:
def process_in_chunks(df, start, end):
    for index, row in df.iloc[start:end].iterrows():
        write_query(create_actor_query, {
            'id': row['id'],
            'name': row['primaryName'],
            'birthYear': row['birthYear'],
            'deathYear': row['deathYear']
        })

        for profession in row['primaryProfession']:
            write_query(create_profession_query, {'profession': profession})
            write_query(create_actor_profession_relationship, {
                'actorId': row['id'],
                'profession': profession
            })

        for title in row['knownForTitles']:
            write_query(create_known_for_title_query, {'titleId': title})
            write_query(create_actor_title_relationship, {
                'actorId': row['id'],
                'titleId': title
            })

In [38]:
%%time
process_in_chunks(df1, 0, 10000)

CPU times: total: 53.2 s
Wall time: 23min 46s


In [39]:
%%time
process_in_chunks(df1, 10000, 20000)

CPU times: total: 20.4 s
Wall time: 21min 27s


In [40]:
%%time
process_in_chunks(df1, 20000, 30000)

CPU times: total: 20.5 s
Wall time: 29min 29s


In [41]:
%%time
process_in_chunks(df1, 30000, 40000)

Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Transaction failed and will be retried in 0.8916326761686947s (Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687))))
Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Transaction failed and will be retried in 1.1166091406293448s (Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687))))
Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Transaction failed and will be retried in 0.9572159893814413s (Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687))))


CPU times: total: 24.9 s
Wall time: 1h 34min 22s


In [20]:
%%time
process_in_chunks(df1, 40000, 50000)

CPU times: total: 24.5 s
Wall time: 52min 57s


In [19]:
%%time
process_in_chunks(df1, 50000, 60000)

Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Transaction failed and will be retried in 1.131045936506472s (Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687))))


CPU times: total: 23.2 s
Wall time: 50min 34s


In [21]:
%%time
process_in_chunks(df1, 60000, 70000)

CPU times: total: 22.3 s
Wall time: 58min 46s


In [22]:
%%time
process_in_chunks(df1, 70000, 80000)

CPU times: total: 24 s
Wall time: 1h 5min 8s


In [23]:
%%time
process_in_chunks(df1, 80000, 90000)

CPU times: total: 24.4 s
Wall time: 1h 14min


In [24]:
%%time
process_in_chunks(df1, 90000, 100000)

CPU times: total: 58.5 s
Wall time: 2h 40min 39s
